[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/chuangtc/ECTSum/blob/main/fine-tune-GPT3.ipynb)


In [1]:
print('Hello, AIDATATOOLS')
!pip install --upgrade openai
import os
os.environ["OPENAI_API_KEY"] = ''

Hello, AIDATATOOLS
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 460.9 kB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 338.3/338.3 KB 1.7 MB/s eta 0:00:0000:0100:01
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 4.6 MB/s eta 0:00:00
  Created wheel for openai: filename=openai-0.26.5-py3-none-any.whl size=67596 sha256=0e25a3d2604207f4a8180bd7b14de1e428d0b46fa94062bbf9910b367aa038e3
  Stored in directory: /Users/jasontgi/Library/Caches/pip/wheels/6b/f6/a3/b6f128b03291d3b9c9b70b45ed4cb5ca32021aa4905b67c761
Successfully built openai
You should consider upgrading via the '/Users/jasontgi/.pyenv/versions/3.9.16/bin/python3.9 -m pip install --upgrade pip' command.


## Do text smmarization, generate training validation data for GPT3

In [ ]:
import json, requests

def parse_line_break(foo): 
  return iter(foo.splitlines())


train_http_path = "https://github.com/chuangtc/ECTSum-GPT3/blob/main/data-bert/train.json?raw=true"  
req = requests.get(train_http_path)

train_json_str_array = []
train_json_str_array = list(parse_line_break(req.text))

train_json =[]
for x in train_json_str_array: 
  tmp_json = json.loads(x)
  tmp_json['prompt'] = 'summarize the text:' + tmp_json.pop('doc')
  tmp_json['completion'] = tmp_json.pop('summaries')
  tmp_json.pop('labels', None)

  train_json.append(tmp_json)

val_http_path = "https://github.com/chuangtc/ECTSum-GPT3/blob/main/data-bert/val.json?raw=true"  
req2 = requests.get(val_http_path)

val_json_str_array = []
val_json_str_array = list(parse_line_break(req2.text))

val_json =[]
for x in val_json_str_array: 
  tmp_json = json.loads(x)
  tmp_json['prompt'] = tmp_json.pop('doc')
  tmp_json['completion'] = tmp_json.pop('summaries')
  tmp_json.pop('labels', None)

  val_json.append(tmp_json)

print('Training examples:')
print(train_json[0])
print(train_json[1])
print('Validation examples:')
print(val_json[0])
print(val_json[1])

In [ ]:
# Writing to sample.json
with open("prepared_train.jsonl", "w") as outfile:
    outfile.write('\n'.join(json.dumps(i) for i in train_json))

# Writing to sample.json
with open("prepared_val.jsonl", "w") as outfile:
    outfile.write('\n'.join(json.dumps(i) for i in val_json))

In [ ]:
!openai api fine_tunes.create -t "prepared_train.jsonl" -v "prepared_val.jsonl" -m ada 

### It takes time to put fine-tune job in queue and training. Let's wait and check the status

In [ ]:
!openai api fine_tunes.follow -i ft-GIfaAtg9FlLFQy8kPd1LOl1p